In [6]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm import tqdm 

from utils.custom_utils import load_var, save_var

## Web Scraping

In [4]:
# df_sort = pd.read_csv("./hunting_sort.csv")
df = pd.read_csv("./dataframe/hunting.csv")

In [7]:
df = load_var("df_hk")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   범죄 유형   0 non-null      float64
 1   지역      3 non-null      object 
 2   위도      0 non-null      float64
 3   경도      0 non-null      float64
 4   중복 여부   6 non-null      object 
 5   퀄리티     0 non-null      float64
 6   제외 여부   0 non-null      float64
 7   기사제목    27 non-null     object 
 8   사건 장소   27 non-null     object 
 9   수사 기관   27 non-null     object 
 10  본문      27 non-null     object 
 11  URL     27 non-null     object 
 12  일자      27 non-null     int64  
 13  언론사     27 non-null     object 
 14  기고자     27 non-null     object 
 15  비고      0 non-null      float64
dtypes: float64(6), int64(1), object(9)
memory usage: 3.5+ KB


### 1st scraping - bs4

In [8]:
import requests
from bs4 import BeautifulSoup

def get_html(url: str) -> str:
    
    if url == float('NaN'):
        return ""
    
    else:
        try:
            response = requests.get(url)
            
            if response.status_code == 200:
                html = response.text
                soup = BeautifulSoup(html, 'html.parser')
                # print(soup)
                # return soup.get_text().replace("\n", " ").replace("  ", "")
                return soup

            else : 
                print(response.status_code)
                return ""            
        except requests.exceptions.InvalidURL as e:
            print(f"Invalid URL: {e}")
            return ""
            
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return ""

In [9]:
def longest_string_length(strings):
    """
    Find the length of the longest string in a list of strings.

    Parameters:
    - strings (list): List of strings.

    Returns:
    - int: Length of the longest string.
    """
    return max(strings, key=len)

In [28]:
# body = soup.select_one('body')
# body_text = body.get_text()
# body_rm = body_text.replace("\n", " ")
# print(body_rm)

           미주한국일보 서울경제TV 시그널 레이디스클래식 디센터 라이프점프 이달의 과학기술인상 서울포럼       주소 : 서울특별시 종로구 율곡로 6 트윈트리타워 B동 14~16층   대표전화 : 02) 724-8600 상호 : 서울경제신문사업자번호 : 208-81-10310대표자 : 손동영등록번호 : 서울 가 00224등록일자 : 1988.05.13 인터넷신문 등록번호 : 서울 아04065 등록일자 : 2016.04.26발행일자 : 2016.04.01발행 ·편집인 : 손동영청소년보호책임자 : 신한수 서울경제의 모든 콘텐트는 저작권법의 보호를 받는 바, 무단 전재·복사·배포 등은 법적 제재를 받을 수 있습니다. Copyright ⓒ Sedaily, All right reserved   서울경제를 팔로우하세요!  서울경제신문         텔레그램 뉴스채널    서울경제 1q60          모바일 버전 보기         피스·상가·토지     건설업계     간접투자     부동산일반       경제 · 금융   경제동향     정책     공기업     경제분석     금융정책     은행     카드     보험     제2금융     재테크     금융가     경제·금융일반       산업   기업     중기·벤처     생활     IT     바이오     산업일반       정치   대통령실     국회·정당·정책     총리실     통일·외교·안보     정치일반         문화 · 스포츠  헬스 라이프 문화 방송·연예 스포츠 자동차      사회   사회일반     전국     사회이슈     피플     영상 · 포토  영상 포토      국제   정치·사회     경제·마켓     기업     인물·화제     국제일반     서경스타  TV·방송 영화 가요 포토      오피니언   사설     사내칼럼     사외칼럼     서경골프  골프일반 생생레슨 10대골프장           공지   [알립니다] '70년 

#### check NA

In [16]:
df[df["URL"].isna()]

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고
162,폭행,제주시,NaN,NaN,미중복,NaN,제외,끊이지 않는 중국인 범죄 노동절 어쩌나,"제주시,중국,제주","연동,경찰,경찰서,제주경찰청",제주에서 중국인이 저지르는 5대 범죄가 끊이지 않고 있어 근본적인 대책 마련이 요구...,NaN,20230428,제민일보,김재연,NaN


#### bs object test

In [73]:
res = get_html(url=df.loc[1, "URL"])

In [76]:
type(res)

bs4.BeautifulSoup

In [78]:
isinstance(res, BeautifulSoup)

True

In [81]:
res_html = res.prettify()

In [84]:
type(res_html)

str

#### scraping

In [85]:
from tqdm import tqdm

DF_MAX = df["URL"].count()

for index in tqdm(range(82, DF_MAX)):
    result = get_html(url=df.loc[index, "URL"])
    if isinstance(result, BeautifulSoup):
        df.loc[index, "soup_html"] = result.prettify()
        df.loc[index, "content_text"] = result.get_text()
    else:
        df.loc[index, "content_text"] = result
    df.loc[index, "content_filtered"] = longest_string_length(df.loc[index, "content"].split("\n\n"))

  2%|███▎                                                                                                                                                                       | 4/208 [00:00<00:29,  6.91it/s]

An error occurred: HTTPSConnectionPool(host='www.fnnews.com', port=443): Max retries exceeded with url: /news/202309111405322672 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))
An error occurred: HTTPSConnectionPool(host='www.fnnews.com', port=443): Max retries exceeded with url: /news/202309110637331373 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))


  5%|████████▏                                                                                                                                                                 | 10/208 [00:06<03:22,  1.02s/it]

An error occurred: Invalid URL 'www.kado.net/news/articleView.html?idxno=1202880': No scheme supplied. Perhaps you meant https://www.kado.net/news/articleView.html?idxno=1202880?


 13%|██████████████████████                                                                                                                                                    | 27/208 [00:17<01:03,  2.84it/s]

...

 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋           | 194/208 [01:16<00:02,  5.26it/s]

403


 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌      | 200/208 [01:22<00:11,  1.45s/it]

An error occurred: ("Connection broken: ConnectionResetError(54, 'Connection reset by peer')", ConnectionResetError(54, 'Connection reset by peer'))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 208/208 [01:26<00:00,  2.40it/s]


In [6]:
url = 'https://www.hankyung.com/society/article/2023041062397'
res = get_html(url=url)

404


#### inspection

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   범죄 유형             291 non-null    object 
 1   지역                107 non-null    object 
 2   위도                35 non-null     float64
 3   경도                35 non-null     float64
 4   중복 여부             191 non-null    object 
 5   퀄리티               47 non-null     object 
 6   제외 여부             291 non-null    object 
 7   기사제목              291 non-null    object 
 8   사건 장소             289 non-null    object 
 9   수사 기관             290 non-null    object 
 10  본문                291 non-null    object 
 11  URL               290 non-null    object 
 12  일자                291 non-null    int64  
 13  언론사               291 non-null    object 
 14  기고자               283 non-null    object 
 15  비고                68 non-null     object 
 16  content           290 non-null    object 
 1

In [89]:
df[df["soup_html"].isna() == True]["soup_html"]

0      NaN
5      NaN
17     NaN
24     NaN
28     NaN
48     NaN
53     NaN
62     NaN
78     NaN
84     NaN
85     NaN
92     NaN
109    NaN
114    NaN
117    NaN
127    NaN
133    NaN
144    NaN
154    NaN
162    NaN
166    NaN
169    NaN
178    NaN
184    NaN
202    NaN
208    NaN
220    NaN
234    NaN
237    NaN
240    NaN
243    NaN
250    NaN
253    NaN
254    NaN
260    NaN
276    NaN
281    NaN
290    NaN
Name: soup_html, dtype: object

#### content filtering test

In [25]:
df.loc[DF_MAX-3, "URL"]

'https://www.hankookilbo.com/News/Read/A2023120411240004749'

In [30]:
text = df.loc[DF_MAX-3, "content"].split("\n\n")

In [42]:
# Example usage:
# string_list = ['apple', 'banana', 'orange', 'kiwi']
longest_length = longest_string_length(text)
print(f"Length of the longest string: {len(longest_length)}")

Length of the longest string: 404


In [44]:
longest_length

'\n서울 강동구(구청장 이수희)는 여성가족부 지정 \'여성친화도시\'에 신규 지정됐다고 4일 밝혔다.\n\'여성친화도시\'는 여성가족부가 여성 권익증진에 노력한 우수 지자체에 부여하는 상이다. 올해 신규 지정된 지자체는 총 15개 시·군·구로 강동구는 서울시내 자치구 중 유일하게 신규 지정됐다.\n강동구는 이번 평가에서 여성 친화형 학습형 일자리 창출과 경력단절 여성 병·의원 취업 연계 프로젝트, 여성 안심 귀갓길 조성사업 등 5대 분야 20개 사업을 제출해 좋은 평가를 받았다. 강동구는 향후 5년 동안 여성 관련 정책 등에 민간 협력 및 컨설팅을 지원 받는다. 이수희 강동구청장은 "앞으로도 여성이 안심하고 살 수 있는 환경을 조성하기 위한 다양한 정책을 발굴해 주민이 체감하는 삶의 질 향상을 위해 최선을 다하겠다"고 말했다.'

In [53]:
for index in tqdm(range(DF_MAX-10, DF_MAX)):
    df.loc[index, "content_filtered"] = longest_string_length(df.loc[index, "content"].split("\n\n"))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1081.79it/s]


In [54]:
df.loc[DF_MAX-10:DF_MAX, "content_filtered"]

280                    KBS / 07235 서울특별시 영등포구 여의공원로 1...
281    \n전북 전주시 덕진구 벚꽃로 54(진북동 417-62)  \n대표전화 : 063-...
282    \nìì¸ ì¡íêµ¬ê° êµ¬ë¯¼ì í¸ìí ê·ê...
283    \nìì°° ì°¸ê°ìë¤ì ì°ë² ì§ì­ ìí ...
284    정찬현 자치경찰과장조국의 광복과 함께 태동한 대한민국의 경찰은 오늘도 사회 질서유지...
285    ¡á ·ÎÄÃÀÎ»çÀÌµå\n\r\n¾È»ê=¹Ú¼ºÈÆ ±âÀÚ pshoon@m...
286    \nìì¸ ê°ëêµ¬ê° ìì¸ìì ì ì¼íê²...
287    \n서울 강동구(구청장 이수희)는 여성가족부 지정 '여성친화도시'에 신규 지정됐다고...
288    올해 전국적으로 신규 지정된 지자체는 15개 시·군·구로 강동구가 서울시에서 유일하...
289    또 음성시니어클럽이 올해 노인일자리 사업 성과보고대회에서 9988시니어봉사회(회장 ...
290                                                   \n
Name: content_filtered, dtype: object

#### clean text

In [114]:
def clean_text(text: str) -> str:
    if isinstance(text, float):
        return ""
    return text.replace("\t", "").replace("\r", "")

In [115]:
df.loc[:, "content"] = df.loc[:, "content"].apply(clean_text)

### 2nd - selenium - 404 error

#### selenium

In [118]:
get_html(df.iloc[0]["URL"])

404


''

In [120]:
%pip install selenium

  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 21.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 32.6 MB/s eta 0:00:00
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ooba 0.0.21 requires huggingface-hub<0.18.0,>=0.17.3, but you have huggingface-hub 0.20.2 which is incompatible.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you ha

In [10]:
from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import InvalidArgumentException

from time import sleep
from tqdm import tqdm

import pandas as pd

In [11]:
driver = webdriver.Chrome()

In [12]:
def get_html_selenium(url: str) -> str:
    try:
        driver.get(url=url)
        body = driver.find_element(by=By.TAG_NAME, value="body")
        # return clean_text(clean_text(body.text.replace("\n", " ")))
        return body.get_attribute('outerHTML')
    
    except InvalidArgumentException as e:
        print(f"Invalid argument exception: {e}")
        return ""

#### empty content 

In [13]:
df_404 = df[df["content"] == ""]

KeyError: 'content'

In [148]:
index_list = df_404.index

In [93]:
df_na = df[df["soup_html"].isna() == True]

In [94]:
index_list = df_na.index

In [95]:
index_list

Index([  0,   5,  17,  24,  28,  48,  53,  62,  78,  84,  85,  92, 109, 114,
       117, 127, 133, 144, 154, 162, 166, 169, 178, 184, 202, 208, 220, 234,
       237, 240, 243, 250, 253, 254, 260, 276, 281, 290],
      dtype='int64')

In [96]:
index_list[3]

24

#### test - 2024.4.7

In [5]:
url = 'https://www.hankyung.com/society/article/2023041062397'

In [20]:
def get_article_text(url, driver):
    driver.get(url=url)
    res_src = driver.page_source
    article_body = driver.find_element(by=By.CLASS_NAME, value="article-body")
    text = article_body.text
    return text

In [9]:
driver.get(url=url)

In [10]:
res_src = driver.page_source

In [11]:
driver.title

'부산 돌려차기 男 나가면 죽인다…구치소 동료 충격 제보 | 한국경제'

In [12]:
article_body = driver.find_element(by=By.CLASS_NAME, value="article-body")

In [13]:
text = article_body.text

In [14]:
print(text)

사건 당시 폐쇄회로(CC)TV에 담긴 폭행 영상. /사진= SBS '그것이 알고싶다' 캡처
일면식 없는 여성을 크게 다치게 한 혐의로 징역형을 선고받고 복역 중인 30대 남성이 피해자에 보복할 계획을 언급했다는 증언이 나왔다.

지난 8일 SBS '그것이 알고싶다'는 '사라진 7분 - 부산 돌려차기 사건의 진실'이라는 제목의 영상을 공개하고, 지난해 5월 22일 발생한 '부산 돌려차기 사건'을 재조명했다.

해당 사건은 부산의 한 오피스텔 공동현관 앞에서 벌어졌다. 20대 여성 A씨가 1층 엘리베이터 버튼을 누르고 기다리던 순간, 이를 뒤따라간 30대 남성 B씨가 돌려차기로 A씨의 후두부를 세게 가격했다.

이에 A씨는 건물 벽면에 머리를 세게 부딪히고 바닥으로 쓰려졌다. B씨는 주먹으로 A씨를 가격하려다 멈칫하더니, A씨가 꿈틀거리자 여러 차례 발길질했고, 결국 A씨는 의식을 잃고 몸이 굳은 채 기절했다.

이 사건으로 A씨는 8주 이상의 치료가 필요한 외상성 두개내출혈과 뇌 손상, 영구장애가 우려되는 다리 마비 등 심각한 상해를 입었다.
가해 남성이 기절한 여성을 어깨에 둘러멘 뒤, CCTV 사각지대로 사라지는 모습. /사진= JTBC '사건반장' 유튜브 영상 캡처
당시 상황이 담긴 폐쇄회로(CC)TV 영상에는 B씨가 A씨의 목덜미 부근을 잡고 끌다가 어깨에 둘러메고 CCTV가 없는 엘리베이터 옆 복도로 사라지는 장면도 담겼다. 이후 B씨는 약 7분간 그곳에 머물다 오피스텔을 빠져나갔다.

7분이라는 시간 동안 성폭행이 있었을 것으로 의심된다는 게 A씨의 판단이다. A씨가 쓰러졌을 당시 병원에 온 그의 언니는 병원에서 동생의 바지를 벗겼을 때 속옷이 없었고, 오른쪽 종아리 한쪽에만 걸쳐져 있었다고 증언했다. 의료진들 역시 성폭행 가능성이 있다는 소견을 냈다.

다만 피해자가 사건 당시 기절하고 기억을 잃은 데다, 경찰과 피해자 모두 사건발생일이 한참 지난 뒤에야 성폭행 가능성을 의심한 상황이라 이를 입증할 증거를 확보하지 못했다.

B씨는 사건 발생 사흘 

#### selenium test

In [142]:
url = df.loc[index_list[3], "URL"]
driver.get(url=url)

In [145]:
type(driver)

selenium.webdriver.chrome.webdriver.WebDriver

In [147]:
res_src = driver.page_source

In [157]:
res_title = driver.title
print(res_title)

길거리서 9살 초등생 성추행한 70대男, 아빠가 잡았다 - 파이낸셜뉴스


In [98]:
result = get_html_selenium(url=df.loc[index_list[3], "URL"])

In [99]:
type(result)

selenium.webdriver.remote.webelement.WebElement

In [106]:
res_html = result.get_attribute('outerHTML')

In [108]:
res_bs = BeautifulSoup(res_html, 'html.parser')

In [141]:
res_bs.prettify()

'<body>\n <!-- Google Tag Manager (noscript) -->\n <noscript>\n  <iframe height="0" src="https://www.googletagmanager.com/ns.html?id=GTM-MB6S9V" style="display:none;visibility:hidden" width="0">\n  </iframe>\n </noscript>\n <!-- End Google Tag Manager (noscript) -->\n <div id="fn_wrap">\n  <!--헤더-->\n  <script>\n   console.log("search_fn");\n    $(function() {\n        $("#main_top").off("click").on(\'click\', function() {\n            //console.log("모바일 메인 검색 상단 클릭"+this.id);\n            search_fn(this.id);\n        });\n\n        $("#main_top_search_input").keypress(function(e) {\n            console.log($(this).attr("data-id"));\n            if (e.which == 13) {\n                var searchId =$(this).attr("data-id");\n                search_fn(searchId);\n            }\n        });\n    });\n\n    function search_fn(id){\n        var searchId = id;\n        console.log("search_fn"+searchId);\n        if ($("#"+searchId+"_search_input").val()==""&&$("#"+searchId+"_search_input").val

In [126]:
res_bs.get_text()

'\n\n\n\n\n\n\n\n\n\n파이낸셜뉴스>\n\n\n\n전체메뉴\n검색\n\n\n\n구독신청\n\n\n\n\n\n\n\n\n\n\n금융·증권\n\n\n금융\n증권\n\n\n\n\n부동산\n\n\n정책\n건설\n철도·항공 ·선박\n부동산 일반\n\n\n\n\n산업·IT\n\n\n산업\n통신·방송\n게임\n인터넷\n블록체인\n의학·과학\n\n\n\n\n경제\n\n\n경제 일반\n생활 경제\n\n\n\n\n정치\n\n\n대통령실\n국회·정당\n북한\n외교·국방\n\n\n\n\n사회\n\n\n사건·사고\n검찰·법원\n행정·지자체\n교육\n전국\n\n\n\n\n국제\n\n\n국제 경제\n국제 정치\n국제 사회\n\n\n\n\n라이프\n\n\n연예\n패션/뷰티\n스포츠\n푸드·리빙\n레저·문화\n\n\n\n\n오피니언\n\n\n사설/칼럼\n사외 칼럼\n\n\n\n\n기획·연재\n\n\nfn파인더\nfn시리즈\n핫이슈+\n\n\n\n\nfnEdition\n\n\n포토\n뉴스레터\n기자ON\nfntv\n신문보기\nfnSurvey\n\n\n\n\n\n\n                            4·10 총선\n                        \n\n후보자\n선거뉴스\n\n닫기\n\n\n회사소개\n광고문의\n제휴문의\n개인정보취급방침\n저작권규약\n이메일주소무단수집거부\n구독신청\n고충처리\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n닫기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n공유하기\n\n공유\n\n\n\n카카오톡\n페이스북\n트위터\n네이버블로그\n주소복사\n\n\n\n\n\n\n\n                        사회\n\n\n                        사건·사고\n\n길거리서 9살 초등생 성추행한 70대男, 아빠가 잡았다\n\n\n\n김수연 기자\n\n\n파이낸셜뉴스\n입력 2023.12.15 06:50\n수정 2023.12.15 17:43\n\n\n\n\

In [127]:
res_split = res_bs.get_text().split("\n\n\n")

#### sort

In [128]:
res_sort = sorted(res_split, key=len)

In [138]:
res_sort[-1]

'\n\'\n                           \n\'\n                       \n\'\n                   \'조폭응징\' 유튜버, 보복 폭행했던 조직폭력배 숨진 채 발견[파이낸셜뉴스] \xa0조직폭력배를 저격하는 내용의 방송을 진행한 유튜버를 보복 폭..파이낸셜뉴스돌아가기울아들램사랑해해외 논문으로 검증된 아동 주의력결핍에 효과적인 1가지 방법 공개울아들램사랑해|SponsoredSponsored더 알아보기돌아가기연세힐치과의원임플란트 1개부터, 전체 임플란트!연세힐치과의원|SponsoredSponsored돌아가기새미래 이석현, 강북을 출사표…[파이낸셜뉴스] 새로운미래 고문인 이석현 전 국회부의장이 22일파이낸셜뉴스돌아가기"아름다운 동행"...대한항공 前여승무원동우회 KASA, 서울역서 급식 봉사 - 파이낸셜뉴스[파이낸셜뉴스] 대한항공 전직 여승무원 동우회인 카사(KASA)는 지난 21일 서울시 용산구 서울역 무료급식소인 \'따스한채움터\'에서 봉사활동을 펼쳤다고 22일 밝혔다.\xa0취약계층 무료급식 제공을 위해 진행된 이날 봉사활동에서는 컵과일과 양말 등의 물품기부와 배식, 서빙 봉사 등도 함께 이뤄졌다.\xa0봉사활동 중에는 8명의 회원들로 구성된 카..파이낸셜뉴스돌아가기\'조수진 리스크’ 제거한 민주…박용진은 끝내 \'비명횡사\'[2024 총선][파이낸셜뉴스] 성범죄자 변호 이력으로 논란이 됐던 조수진 변호사가 서울 강북을 후보에서 사퇴하면서 더불어민주당이 부담을 한결 내려놓게 됐다. 다시금 공석이 된 강북을에는 \'친명계\' 한민수 대변인이 공천됐다. 각종 우여곡절 끝에 민주당이 한 대변인을 공천했으나 일각에서는 사천 문제와 함께 총선 이후 갈등의 새로운 씨앗이 될 것이라는 예측..파이낸셜뉴스돌아가기\'7살 제자 일가족 몰살\' 혐의 태권도 관장, 모든 것이 거짓 "아내도 당했다"[파이낸셜뉴스] \xa0지난달 호주 시드니에서 한인 일가족 3명을 살해한 혐의를 받는 태권도 관장의 만행이 드러났다. 17일 S

In [137]:
res_sort[-2]

'\n울아들램사랑해해외 논문으로 검증된 아동 주의력결핍에 효과적인 1가지 방법 공개울아들램사랑해|SponsoredSponsored더 알아보기돌아가기연세힐치과의원임플란트 1개부터, 전체 임플란트!연세힐치과의원|SponsoredSponsored돌아가기연세힐치과의원전체 임플란트 해야 하는데 부담된다면?연세힐치과의원|SponsoredSponsored돌아가기JUVIS34kg 감량된 내 모습을보니 월 16만원이 전혀 아깝지 않았어요JUVIS|SponsoredSponsored돌아가기울아들램사랑해ADHD 카페에서 난리난 ADHD 개선법 1가지울아들램사랑해|SponsoredSponsored더 알아보기돌아가기Rg3 더루트"면역력, 혈압" 결정짓는 "이 성분"을 가득 담아.. 카이스트 개발 화제!Rg3 더루트|SponsoredSponsored돌아가기강남애프터치과전체임플란트 필요하신 분이라면?대한치과의사협회 의료광고 심의필 제231127-중-28744호강남애프터치과|SponsoredSponsored가격 받기돌아가기원데이 임플란트빠진 치아가 많나요? 전체 임플란트로 치료하세요전체 임플란트, 의식하진정법 강남역 11번 출구, 원데이치과의원원데이 임플란트|SponsoredSponsored더 알아보기돌아가기임플란트국산 임플란트 개당 "35만원" 특가 이벤트!공중파 방송에서 인정한 의료진! 지르코니아+맞춤형지대주 포함 금액!임플란트|SponsoredSponsored더 알아보기돌아가기장덕한방병원어깨통증 수술없이 당일치료하세요30년 양한방 비수술 치료 노하우로 어깨통증 근본원인부터 해결합니다.장덕한방병원|SponsoredSponsored더 알아보기돌아가기신(新) 의료기술 등재 - 자가골수 줄기세포 치료"자가골수 줄기세포" 신 의료기술 등재! 수술없이 치료가능신(新) 의료기술 등재 - 자가골수 줄기세포 치료|SponsoredSponsored더 알아보기돌아가기Rg3 더루트"혈액순환 잘되네요" 손발저림까지 싹~\'이것\', 하루 1번이면 된다!카이스트 박사가 24년간 연구한 특이사포닌 Rg3 성분을

In [135]:
res_sort[-3]

'발아픈사람이야기족저근막염 극복한 딱 1가지 방법족저근막염 때문에 폐업까지 하게 된 남자가 이 방법을 알고 족저근막염을 극복하게 되는데...발아픈사람이야기|SponsoredSponsored더 알아보기돌아가기신(新) 의료기술 등재 - 자가골수 줄기세포 치료무릎통증 "줄기세포" 치료, 수술없이 당일 일상복귀가능신(新) 의료기술 등재 - 자가골수 줄기세포 치료|SponsoredSponsored더 알아보기돌아가기공동투자그룹(국내 트레이딩)주식\xa0 매수, 매도 타이밍(전문 주식 트레이더 종목 선정 이번주만 무료)공동투자그룹(국내 트레이딩)|SponsoredSponsored더 알아보기돌아가기Navy.Quest마우스가 있다면 밤새워 즐기게 되는 게임. 설치 없이 바로 실행. 무료로 플레이.Navy.Quest|SponsoredSponsored돌아가기Mech Arena- Free to Play마우스 하나로 밤새는 게임!실시간 5대5 메카닉 대전Mech Arena- Free to Play|SponsoredSponsored다운로드돌아가기오직팩트만관절마다 붓고 아프다면 "이것" 결핍 의심알아야 건강할 수 있습니다.오직팩트만|SponsoredSponsored더 알아보기돌아가기새벽 공원 벤치서 웅크린 채 잠든 여학생...지켜본 시민의 행동 [따뜻했슈]파이낸셜뉴스돌아가기여친 190번 찔러 살해한 동거남, 이름·신상공개됐다파이낸셜뉴스돌아가기'

In [132]:
res_sort[-4]

'  이준석 기자\n \n\n[파이낸셜뉴스] 길거리에서 9살 초등학생을 성추행한 70대 남성이 경찰에 붙잡혔다. 이 남성은 피해 아동의 아버지에게 붙잡혀 현행범으로 체포됐다.\n\n14일 경찰에 따르면 서울경찰청 여성청소년범죄수사대는 전날 성폭력범죄의 처벌 등에 관한 특례법(성폭력처벌법) 위반 혐의로 70대 남성 A씨에 대한 구속영장을 신청했다고 밝혔다.\n\nA씨는 지난 12일 오후 8시30분께 서울 동작구의 한 길거리에서 9살 여자 초등학생을 불러 세운 뒤 신체 등을 만진 혐의를 받는다.\n\n피해 아동은 자신의 피해 사실을 아버지인 B씨에게 알렸다.\n 이러한 사실을 알게 된 B씨는 직접 A씨를 붙잡은 것으로 전해졌다.\n\nB씨에게 붙잡힌 A씨는 출동한 경찰에 의해 현행범으로 체포됐다.\n\n경찰 관계자는 "구체적인 내용은 확인이 어렵다"면서도 "(A씨에 대한) 구속영장을 신청한 건 맞다"고 전했다.\n\nnewssu@fnnews.com 김수연 기자 '

In [139]:
res_sort[-5]

'\n여러분이 생각하는 4.10 총선 핵심 의제는?\n\n2024년 총선이 한 달도 채 남지 않았습니다.국회의원 선거는 단순한 정치적 이익을 넘어국민의 삶을 실질적으로 개선하는정책 공약의 경연장이 되어야 할 텐데요.이에 현실성 있고 효과적인 의제들이 더욱 많이 다뤄지길 바라는 유권자들의 목소리가 큽니다.그렇다면 여러분이 생각하기에 가장 중요하다고 생각되는 핵심 의제는 무엇인가요?\n\n투표기간 : 2024-03-18 ~ 2024-04-09\n                                         \n투표하러 가기'

In [111]:
res_filtered = longest_string_length(res_bs.get_text().split("\n\n"))

In [112]:
res_filtered

'\n\'\n                           \n\'\n                       \n\'\n                   \'조폭응징\' 유튜버, 보복 폭행했던 조직폭력배 숨진 채 발견[파이낸셜뉴스] \xa0조직폭력배를 저격하는 내용의 방송을 진행한 유튜버를 보복 폭..파이낸셜뉴스돌아가기울아들램사랑해해외 논문으로 검증된 아동 주의력결핍에 효과적인 1가지 방법 공개울아들램사랑해|SponsoredSponsored더 알아보기돌아가기연세힐치과의원임플란트 1개부터, 전체 임플란트!연세힐치과의원|SponsoredSponsored돌아가기새미래 이석현, 강북을 출사표…[파이낸셜뉴스] 새로운미래 고문인 이석현 전 국회부의장이 22일파이낸셜뉴스돌아가기"아름다운 동행"...대한항공 前여승무원동우회 KASA, 서울역서 급식 봉사 - 파이낸셜뉴스[파이낸셜뉴스] 대한항공 전직 여승무원 동우회인 카사(KASA)는 지난 21일 서울시 용산구 서울역 무료급식소인 \'따스한채움터\'에서 봉사활동을 펼쳤다고 22일 밝혔다.\xa0취약계층 무료급식 제공을 위해 진행된 이날 봉사활동에서는 컵과일과 양말 등의 물품기부와 배식, 서빙 봉사 등도 함께 이뤄졌다.\xa0봉사활동 중에는 8명의 회원들로 구성된 카..파이낸셜뉴스돌아가기\'조수진 리스크’ 제거한 민주…박용진은 끝내 \'비명횡사\'[2024 총선][파이낸셜뉴스] 성범죄자 변호 이력으로 논란이 됐던 조수진 변호사가 서울 강북을 후보에서 사퇴하면서 더불어민주당이 부담을 한결 내려놓게 됐다. 다시금 공석이 된 강북을에는 \'친명계\' 한민수 대변인이 공천됐다. 각종 우여곡절 끝에 민주당이 한 대변인을 공천했으나 일각에서는 사천 문제와 함께 총선 이후 갈등의 새로운 씨앗이 될 것이라는 예측..파이낸셜뉴스돌아가기\'7살 제자 일가족 몰살\' 혐의 태권도 관장, 모든 것이 거짓 "아내도 당했다"[파이낸셜뉴스] \xa0지난달 호주 시드니에서 한인 일가족 3명을 살해한 혐의를 받는 태권도 관장의 만행이 드러났다. 17일 S

In [151]:
df.loc[index_list[3], "content"] = result

#### scraping

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   범죄 유형   0 non-null      float64
 1   지역      3 non-null      object 
 2   위도      0 non-null      float64
 3   경도      0 non-null      float64
 4   중복 여부   6 non-null      object 
 5   퀄리티     0 non-null      float64
 6   제외 여부   0 non-null      float64
 7   기사제목    27 non-null     object 
 8   사건 장소   27 non-null     object 
 9   수사 기관   27 non-null     object 
 10  본문      27 non-null     object 
 11  URL     27 non-null     object 
 12  일자      27 non-null     int64  
 13  언론사     27 non-null     object 
 14  기고자     27 non-null     object 
 15  비고      0 non-null      float64
dtypes: float64(6), int64(1), object(9)
memory usage: 3.5+ KB


In [189]:
for index in tqdm(index_list):
    result = get_html_selenium(url=df.loc[index, "URL"])
    df.loc[index, "selenium_html"] = result

  0%|                                                                                                                                                                                    | 0/38 [00:00<?, ?it/s]/var/folders/r0/w084nh2s0v3fstg3n70027500000gn/T/ipykernel_85786/1728345963.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<body>
	<div id="wrap" class="view">

		<!-- 스킵네비게이션 -->
		<div class="skip_navi">
			<a href="#container">본문 바로가기</a>
		</div>
		<!-- //스킵네비게이션 -->

		<header class="header-main ver2022">
            <div class="top-util-area">
	<div class="top-util-inner">
		<!-- <div class="util-items lt">
			<div class="item today-info"></div>
			<div class="item hk-slogan"> 성공을 부르는 습관</div>
		</div> -->
		<div class="util-items rt">
			<div class="item">
				<div class="gnb-dropdown type-click" tabindex="0">
					<strong class="gnb-dropdown-toggle"><span class="txt-en">Family Site</span></strong>


Invalid argument exception: Message: invalid argument
  (Session info: chrome=123.0.6312.59)
Stacktrace:
0   chromedriver                        0x0000000102b483e8 chromedriver + 4326376
1   chromedriver                        0x0000000102b408b0 chromedriver + 4294832
2   chromedriver                        0x000000010276beec chromedriver + 278252
3   chromedriver                        0x0000000102755748 chromedriver + 186184
4   chromedriver                        0x0000000102753e9c chromedriver + 179868
5   chromedriver                        0x00000001027547fc chromedriver + 182268
6   chromedriver                        0x000000010276e620 chromedriver + 288288
7   chromedriver                        0x00000001027e7b14 chromedriver + 785172
8   chromedriver                        0x00000001027e74f8 chromedriver + 783608
9   chromedriver                        0x00000001027a34e4 chromedriver + 505060
10  chromedriver                        0x00000001027a3f5c chromedriver + 507740
11

 29%|█████████████████████████████████████████████████▌                                                                                                                         | 11/38 [00:33<00:33,  1.26s/it]

Invalid argument exception: Message: invalid argument
  (Session info: chrome=123.0.6312.59)
Stacktrace:
0   chromedriver                        0x0000000102b483e8 chromedriver + 4326376
1   chromedriver                        0x0000000102b408b0 chromedriver + 4294832
2   chromedriver                        0x000000010276beec chromedriver + 278252
3   chromedriver                        0x0000000102755748 chromedriver + 186184
4   chromedriver                        0x0000000102753e9c chromedriver + 179868
5   chromedriver                        0x00000001027547fc chromedriver + 182268
6   chromedriver                        0x000000010276e620 chromedriver + 288288
7   chromedriver                        0x00000001027e7b14 chromedriver + 785172
8   chromedriver                        0x00000001027e74f8 chromedriver + 783608
9   chromedriver                        0x00000001027a34e4 chromedriver + 505060
10  chromedriver                        0x00000001027a3f5c chromedriver + 507740
11

 50%|█████████████████████████████████████████████████████████████████████████████████████▌                                                                                     | 19/38 [00:58<00:45,  2.42s/it]

...

 74%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                             | 28/38 [02:44<03:25, 20.52s/it]

Invalid argument exception: Message: invalid argument
  (Session info: chrome=123.0.6312.59)
Stacktrace:
0   chromedriver                        0x0000000102b483e8 chromedriver + 4326376
1   chromedriver                        0x0000000102b408b0 chromedriver + 4294832
2   chromedriver                        0x000000010276beec chromedriver + 278252
3   chromedriver                        0x0000000102755748 chromedriver + 186184
4   chromedriver                        0x0000000102753e9c chromedriver + 179868
5   chromedriver                        0x00000001027547fc chromedriver + 182268
6   chromedriver                        0x000000010276e620 chromedriver + 288288
7   chromedriver                        0x00000001027e7b14 chromedriver + 785172
8   chromedriver                        0x00000001027e74f8 chromedriver + 783608
9   chromedriver                        0x00000001027a34e4 chromedriver + 505060
10  chromedriver                        0x00000001027a3f5c chromedriver + 507740
11

 79%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                    | 30/38 [02:48<01:42, 12.86s/it]

Invalid argument exception: Message: invalid argument
  (Session info: chrome=123.0.6312.59)
Stacktrace:
0   chromedriver                        0x0000000102b483e8 chromedriver + 4326376
1   chromedriver                        0x0000000102b408b0 chromedriver + 4294832
2   chromedriver                        0x000000010276beec chromedriver + 278252
3   chromedriver                        0x0000000102755748 chromedriver + 186184
4   chromedriver                        0x0000000102753e9c chromedriver + 179868
5   chromedriver                        0x00000001027547fc chromedriver + 182268
6   chromedriver                        0x000000010276e620 chromedriver + 288288
7   chromedriver                        0x00000001027e7b14 chromedriver + 785172
8   chromedriver                        0x00000001027e74f8 chromedriver + 783608
9   chromedriver                        0x00000001027a34e4 chromedriver + 505060
10  chromedriver                        0x00000001027a3f5c chromedriver + 507740
11

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [04:23<00:00,  6.93s/it]


In [16]:
df_len = len(df["URL"])

In [21]:
for index in tqdm(range(df_len)):
    # print(index)
    # result = get_html_selenium(url=df.loc[index, "URL"])
    result = get_article_text(url=df.loc[index, "URL"], driver=driver)
    df.loc[index, "text"] = result

  0%|                                                                                                                               | 0/27 [00:00<?, ?it/s]/var/folders/r0/w084nh2s0v3fstg3n70027500000gn/T/ipykernel_43935/2939051705.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '사건 당시 폐쇄회로(CC)TV에 담긴 폭행 영상. /사진= SBS '그것이 알고싶다' 캡처
일면식 없는 여성을 크게 다치게 한 혐의로 징역형을 선고받고 복역 중인 30대 남성이 피해자에 보복할 계획을 언급했다는 증언이 나왔다.

지난 8일 SBS '그것이 알고싶다'는 '사라진 7분 - 부산 돌려차기 사건의 진실'이라는 제목의 영상을 공개하고, 지난해 5월 22일 발생한 '부산 돌려차기 사건'을 재조명했다.

해당 사건은 부산의 한 오피스텔 공동현관 앞에서 벌어졌다. 20대 여성 A씨가 1층 엘리베이터 버튼을 누르고 기다리던 순간, 이를 뒤따라간 30대 남성 B씨가 돌려차기로 A씨의 후두부를 세게 가격했다.

이에 A씨는 건물 벽면에 머리를 세게 부딪히고 바닥으로 쓰려졌다. B씨는 주먹으로 A씨를 가격하려다 멈칫하더니, A씨가 꿈틀거리자 여러 차례 발길질했고, 결국 A씨는 의식을 잃고 몸이 굳은 채 기절했다.

이 사건으로 A씨는 8주 이상의 치료가 필요한 외상성 두개내출혈과 뇌 손상, 영구장애가 우려되는 다리 마비 등 심각한 상해를 입었다.
가해 남성이 기절한 여성을 어깨에 둘러멘 뒤, CCTV 사각지대로 사라지는 모습. /사진= JTBC '사건반장' 유튜브 영상 캡처
당시 상황이 담긴 폐쇄회로(CC)TV 영상에는 B씨가 A씨의 목덜미 

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   범죄 유형   0 non-null      float64
 1   지역      3 non-null      object 
 2   위도      0 non-null      float64
 3   경도      0 non-null      float64
 4   중복 여부   6 non-null      object 
 5   퀄리티     0 non-null      float64
 6   제외 여부   0 non-null      float64
 7   기사제목    27 non-null     object 
 8   사건 장소   27 non-null     object 
 9   수사 기관   27 non-null     object 
 10  본문      27 non-null     object 
 11  URL     27 non-null     object 
 12  일자      27 non-null     int64  
 13  언론사     27 non-null     object 
 14  기고자     27 non-null     object 
 15  비고      0 non-null      float64
 16  text    27 non-null     object 
dtypes: float64(6), int64(1), object(10)
memory usage: 3.7+ KB


In [25]:
df.iloc[1]["text"]

'/사진=게티이미지뱅크\n일면식도 없는 여성의 집에 무단 침입해 주먹을 휘두른 60대 남성이 경찰에 붙잡혔다.\n\n전북 김제경찰서는 여성 집에 무단으로 들어가 폭력을 행사한 혐의(주거침입 등)로 60대 A씨에 대해 구속영장을 신청했다고 13일 밝혔다.\n\n경찰에 따르면 A씨는 지난 11일 오후 2시께 김제시의 한 아파트에 침입해 60대 여성 B씨를 주먹으로 여러 차례 때린 혐의를 받고 있다.\n\nA씨에게 폭행당한 B씨는 소리를 지르며 아파트 복도로 뛰어나왔고, 이를 목격한 아파트 동장이 A씨를 붙잡아 경찰에 신고했다.\n\n경찰 조사 결과 A씨는 B씨와 일면식이 없는 사이로, 문이 열려 있는 피해자의 집 현관에 여성 신발만 놓여있자 범행한 것으로 파악됐다.\n\n경찰 조사에서 A씨는 "여자 혼자 있는 것 같아서 이야기 좀 하려고 그랬다"고 진술한 것으로 알려졌다.\n\n경찰은 성범죄 등 다른 피해는 없었지만, 죄질이 나빠 A씨에 대해 구속영장을 신청했고, 구체적인 범행 동기 등을 조사한다는 방침이다.\n\n이보배 한경닷컴 객원기자 newsinfo@hankyung.com'

In [23]:
save_var(df, "df_hk_text")

In [136]:
df["content"] = df[df["content"] == ""]["URL"].apply(get_html_selenium)

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=122.0.6261.112)
Stacktrace:
0   chromedriver                        0x0000000103084508 chromedriver + 3966216
1   chromedriver                        0x000000010307ca94 chromedriver + 3934868
2   chromedriver                        0x0000000102cffbf4 chromedriver + 277492
3   chromedriver                        0x0000000102ce994c chromedriver + 186700
4   chromedriver                        0x0000000102ce7d2c chromedriver + 179500
5   chromedriver                        0x0000000102ce86f8 chromedriver + 182008
6   chromedriver                        0x0000000102d021b8 chromedriver + 287160
7   chromedriver                        0x0000000102d7b20c chromedriver + 782860
8   chromedriver                        0x0000000102d7abf0 chromedriver + 781296
9   chromedriver                        0x0000000102d36fb0 chromedriver + 503728
10  chromedriver                        0x0000000102d37a28 chromedriver + 506408
11 

#### driver close

In [156]:
driver.close()
driver.quit()

#### content inspection

In [158]:
df[df["content"] == ""]

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,content
62,헌팅,NaN,NaN,NaN,미중복,NaN,제외,"기강해이 해경, 국감서 주요 이슈로 '눈총'","고흥,광주갑,강,경기,장흥ㆍ,전남","해경,해경청,해양경찰청장,더불어민주당,중부해경청,해양경찰청",해경의 무너진 공직기강이 국정감사의 주요 이슈가 됐다. \n\n22일 소병훈(더불어...,www.joongboo.com/news/articleView.html?idxno=3...,20231022,중부일보,김상윤,NaN,
92,헌팅,NaN,NaN,NaN,중복,NaN,포함,음주운전 적발 후 모르는 10대 여성 추행 혐의 40대 징역 1년 선고,-,"원주시,원주,검찰,춘천지법,원주지원,형사1단독",음주단속에 걸린지 1시간 30여분 만에 길거리에서 마주친 일면식도 없는 10대 여성...,www.kado.net/news/articleView.html?idxno=1202880,20230910,강원도민일보,신정은,NaN,
162,폭행,제주시,NaN,NaN,미중복,NaN,제외,끊이지 않는 중국인 범죄 노동절 어쩌나,"제주시,중국,제주","연동,경찰,경찰서,제주경찰청",제주에서 중국인이 저지르는 5대 범죄가 끊이지 않고 있어 근본적인 대책 마련이 요구...,NaN,20230428,제민일보,김재연,NaN,
208,미행,NaN,NaN,NaN,NaN,NaN,제외,'진천형 여성친화도시' 조성 기반 구축에 208억원 투입,"진천,중부매일,진,진천군","진천군,여성단체협의회",［중부매일 송창희 기자］ 진천군이 여성친화도시 지정 4년 차를 맞아 '민ㆍ관이 공감...,www.jbnews.com/news/articleView.html?idxno=142...,20240221,중부매일,송창희 기자,NaN,
237,미행,NaN,NaN,NaN,NaN,NaN,제외,"진천군, 여성친화도시 조성 제3기 군민참여단 모집","충북,중부매일,진천군,군민참여단원,단원","가족,진천군,군청,진천군청,군민참여단,여성복",［중부매일 송창희 기자］ 진천군이 오는 28일까지 여성친화도시 조성을 위한 제3기 ...,www.jbnews.com/news/articleView.html?idxno=142...,20240201,중부매일,송창희 기자,NaN,
243,미행,NaN,NaN,NaN,NaN,NaN,제외,"인천 계양구, '여성친화도시' 조성 박차","서울,인천,계양구,인천시","계양구,여성가족부",인천시 계양구가 지난 23일 서울 포스트타워에서 여성가족부와 여성친화도시 신규 지정...,www.joongboo.com/news/articleView.html?idxno=3...,20240124,중부일보,김상윤,NaN,


In [159]:
index_list

Index([  0,  17,  28,  48,  62,  92, 109, 117, 127, 133, 162, 166, 178, 202,
       208, 237, 243, 254, 260, 276, 290],
      dtype='int64')

#### InvalidArgumentException

In [162]:
driver = webdriver.Chrome()

Invalid argument exception: Message: invalid argument
  (Session info: chrome=122.0.6261.112)
Stacktrace:
0   chromedriver                        0x00000001026b8508 chromedriver + 3966216
1   chromedriver                        0x00000001026b0a94 chromedriver + 3934868
2   chromedriver                        0x0000000102333bf4 chromedriver + 277492
3   chromedriver                        0x000000010231d94c chromedriver + 186700
4   chromedriver                        0x000000010231bd2c chromedriver + 179500
5   chromedriver                        0x000000010231c6f8 chromedriver + 182008
6   chromedriver                        0x00000001023361b8 chromedriver + 287160
7   chromedriver                        0x00000001023af20c chromedriver + 782860
8   chromedriver                        0x00000001023aebf0 chromedriver + 781296
9   chromedriver                        0x000000010236afb0 chromedriver + 503728
10  chromedriver                        0x000000010236ba28 chromedriver + 506408
1

In [164]:
url = df.loc[62, "URL"]
print(url)
# result = get_html_selenium(url=url)
# df.loc[index, "content"] = result

www.joongboo.com/news/articleView.html?idxno=363616727


ah... there's no https:// in url

In [169]:
df.loc[0, "URL"]

'https://www.hankyung.com/article/2024022231661'

In [172]:
df[df["URL"].str.split(".").str[0] == "www"]

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,content


In [171]:
df["URL"] = df[df["URL"].str.split(".").str[0] == "www"]["URL"].apply(lambda x: "https://" + x)


In [176]:
df_https = df[df["content"] == ""]
index_list = df_https.index
index_list

Index([62, 92, 162, 208, 237, 243], dtype='int64')

In [177]:
for index in tqdm(index_list):
    result = get_html_selenium(url=df.loc[index, "URL"])
    df.loc[index, "content"] = result

 33%|███▎      | 2/6 [00:09<00:19,  4.92s/it]

Invalid argument exception: Message: invalid argument: missing command parameters
Stacktrace:
0   chromedriver                        0x00000001026b8508 chromedriver + 3966216
1   chromedriver                        0x00000001026b0a94 chromedriver + 3934868
2   chromedriver                        0x0000000102333da0 chromedriver + 277920
3   chromedriver                        0x000000010238ee90 chromedriver + 650896
4   chromedriver                        0x000000010238e350 chromedriver + 648016
5   chromedriver                        0x0000000102306368 chromedriver + 90984
6   chromedriver                        0x000000010267d6f0 chromedriver + 3725040
7   chromedriver                        0x0000000102681be4 chromedriver + 3742692
8   chromedriver                        0x00000001026661d8 chromedriver + 3629528
9   chromedriver                        0x00000001026826e0 chromedriver + 3745504
10  chromedriver                        0x0000000102659550 chromedriver + 3577168
11  chrom

100%|██████████| 6/6 [00:17<00:00,  2.90s/it]


In [179]:
driver.close()
driver.quit()

In [178]:
df[df["content"] == ""]

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,content
162,폭행,제주시,NaN,NaN,미중복,NaN,제외,끊이지 않는 중국인 범죄 노동절 어쩌나,"제주시,중국,제주","연동,경찰,경찰서,제주경찰청",제주에서 중국인이 저지르는 5대 범죄가 끊이지 않고 있어 근본적인 대책 마련이 요구...,NaN,20230428,제민일보,김재연,NaN,


#### save a text to a Markdown file

In [26]:
def save_to_markdown(content, file_path):
    """
    Save content to a Markdown file.

    Parameters:
    - content (str): The content to be saved.
    - file_path (str): The path to the Markdown file.
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

In [8]:
df.iloc[4]["URL"]

'http://news.moneytoday.co.kr/view/mtview.php?no=2024020518521145825&type=2'

In [63]:
df.iloc[4]["content"]

'\n\n\n\'묻지마 폭행\'에 행인 중태…20대男 구속영장 2회 기각, 왜? - 머니투데이\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문영역 바로가기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n로그인\n\n\n\n\n\n \n\n\n\n\n\n\n머니투데이\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n전체\n\n\n\n뉴스\n\n\n경제정책\n\n\n증권\n\n\n정치\n\n\n부동산\n\n\n국제경제\n\n\n사회\n\n\n산업\n\n\nIT\n\n\n금융\n\n\n벤처·스타트업\n\n\n블록체인\n\n\n문화\n\n\n포토\n\n포토\n\n\n더차트\n\n\n더그래픽\n\n\n\n\n영상\n\n\nMT 영상\n\n\n더영상\n\n\n\n\n전체기사\n\n\n\n\n\n증권\n\n\n뉴스\n\n\n시세/공시\n\n코스피\n\n코스닥\n\n코넥스\n\n상승/하락\n\n상하한가\n\n시가총액\n\n거래량상위\n\n신규상장\n\n주요공시\n\n\n\n\n포인트&전략\n\n\n펀드\n\n펀드IR\n\n국내펀드\n\n해외펀드\n\n유형종합\n\n\n\n\n종목대해부\n\n\n베스트리포트\n\n\nMY스톡\n\n\n\n\n\n정치\n\n\n\n법률\n\n\n\n유니콘팩토리\n\n\n\n\n헬스·바이오\n\n\n최신기사\n\n\n\n\n\n칼럼\n\n\n데스크칼럼\n\n\n사내칼럼\n\n\n현장리포트\n\n\n사외칼럼\n\n\nSUCCESS\n\n\n\n\n\n\n\n\n\n\n연예\n\n\n연예\n\n\n스포츠\n\n\n\n\n\n이슈\n

In [61]:
def get_markdown(index):
    content = df.iloc[index]["content_filtered"]
    title = f'{df.iloc[index]["일자"]}_{df.iloc[index]["언론사"]}'
    save_to_markdown(content=content, file_path=f"./{title}.md")

In [62]:
get_markdown(4)

In [28]:
save_to_markdown(text, "hk_test.md")

### to CSV

In [190]:
df.to_csv("./hunting_content.csv")

In [2]:
import pandas as pd

df = pd.read_csv("./hunting_content.csv")

## Location Information

In [3]:
import numpy as np

In [4]:
df_loc = df[np.isnan(df["위도"]) == False][["지역", "위도", "경도"]]

In [5]:
for item in df_loc["지역"]:
    print(f"\'{item}\'", end=", ")

'서구 초장동', '중구 대청동1가', '종로구 인의동', '동작구 노량진동', '남양주시 금곡동', '부천시 중동', '부산진구 부암동', '강남구 삼성동', '북구 풍향동', '서초구서초동', '강남구 삼성동', '경기 성남시 삼평동', '원주시 무실동', '울산 남구 신정동', '제주 서귀포시 서흥동', '광주 서구 치평동', '제주시 연동', '경기 의왕시 의천동', '서울시 노원구 공릉동', '서울시', '부산시 동구', '경기도 고양시 덕양구', '대구시 동구', '인천시 부평구', '부산시', '대구시', '대구광역시청 동인청사', '부산 중구청', '대구', '부산 서구 구청', '부산
(부산광역시 
부산진구 서면
(부전동) 
한 오피스텔 
공동현관)', '부산', '논산
논산중앙초등학교 (충청남도 논산시)', '논산', '논산', 

## Restoring URL 

In [219]:
df_org = pd.read_csv("./hunting.csv")

In [220]:
df["URL"] = df_org["URL"]

In [221]:
df.iloc[4]["URL"]

'http://news.moneytoday.co.kr/view/mtview.php?no=2024020518521145825&type=2'

## Process

### import

In [109]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var

In [41]:
from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import InvalidArgumentException, WebDriverException

from time import sleep
from tqdm import tqdm

import pandas as pd

In [30]:
driver = webdriver.Chrome()

In [31]:
type(driver)

selenium.webdriver.chrome.webdriver.WebDriver

### def: get_html_selenium()

In [39]:
def get_html_selenium(
    url: str,
    driver: webdriver.chrome.webdriver.WebDriver
) -> str:
    try:
        driver.get(url=url)
        body = driver.find_element(by=By.TAG_NAME, value="body")
        # return clean_text(clean_text(body.text.replace("\n", " ")))
        return body.get_attribute('outerHTML')
    
    except InvalidArgumentException as e:
        print(f"Invalid argument exception: {e}")
        return ""

    except WebDriverException as e:
        print(f"WebDriverException: {e}")
        return ""


### def: get_article_text()

In [34]:
def get_article_text(
    url, 
    driver: webdriver.chrome.webdriver.WebDriver
):
    driver.get(url=url)
    res_src = driver.page_source
    article_body = driver.find_element(by=By.CLASS_NAME, value="article-body")
    text = article_body.text
    return text

### data

In [7]:
df = pd.read_excel("./dataframe/EDITED_1_바바리맨_자연어_한국경제_2013.1.1~2022.3.7 (1).xlsx")

#### inspection

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   범죄 유형   0 non-null      float64
 1   지역      0 non-null      float64
 2   위도      0 non-null      float64
 3   경도      0 non-null      float64
 4   중복 여부   0 non-null      float64
 5   퀄리티     0 non-null      float64
 6   제외 여부   0 non-null      float64
 7   기사제목    377 non-null    object 
 8   사건 장소   328 non-null    object 
 9   수사 기관   377 non-null    object 
 10  본문      377 non-null    object 
 11  URL     209 non-null    object 
 12  일자      377 non-null    int64  
 13  언론사     377 non-null    object 
 14  기고자     286 non-null    object 
 15  비고      0 non-null      float64
dtypes: float64(8), int64(1), object(7)
memory usage: 47.2+ KB


#### features not having URL

In [13]:
df_columns = list(df.columns)
print(df_columns)

['범죄 유형', '지역', '위도', '경도', '중복 여부', '퀄리티', '제외 여부', '기사제목', '사건 장소', '수사 기관', '본문', 'URL', '일자', '언론사', '기고자', '비고']


In [17]:
column_filter = [df_columns[7], df_columns[11]]
column_filter

['기사제목', 'URL']

In [22]:
# df[df[df_columns[7]].isna() == False && df_columns[11].isna() == True]

df_filtered = df[(df[df_columns[7]].notna()) & (df[df_columns[11]].isna())]

In [23]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, 198 to 376
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   범죄 유형   0 non-null      float64
 1   지역      0 non-null      float64
 2   위도      0 non-null      float64
 3   경도      0 non-null      float64
 4   중복 여부   0 non-null      float64
 5   퀄리티     0 non-null      float64
 6   제외 여부   0 non-null      float64
 7   기사제목    168 non-null    object 
 8   사건 장소   150 non-null    object 
 9   수사 기관   168 non-null    object 
 10  본문      168 non-null    object 
 11  URL     0 non-null      object 
 12  일자      168 non-null    int64  
 13  언론사     168 non-null    object 
 14  기고자     82 non-null     object 
 15  비고      0 non-null      float64
dtypes: float64(8), int64(1), object(7)
memory usage: 22.3+ KB


In [24]:
df_filtered.head()

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고
198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"인천초등생살인사건, 주범이 공범보다 형량이 낮은 이유는?","인천초,김양,고구려,청원,인천초등생,인천초등,인천","서울고등법원,정부,청와대,검찰,한경닷컴",지난 15일 서울고등법원에서 지난해 온 국민을 충격으로 빠뜨린 사건 인천초등\n학생...,NaN,20180126,한국경제,NaN,NaN
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,논란 '인천 여고생 폭행' 가해자 4명 이름-사진까지 노출 신상 털기 '확산',인천시,"정보통신,한경닷컴,경찰,남동구","네티즌들의 공분을 일으킨 ‘여고생 집단 폭행사건’과 관련, 가해자\n 4명의 신상이...",NaN,20180109,한국경제,NaN,NaN
200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'인천여고생 집단폭행에 성매매 강요' 가해자 전원 체포 구출 도운 성매수男 처벌은?,"오산휴게소,부산,삭제,인천,봉담,인천시,수도권","남동경찰서,인천지방경찰청,한경닷컴,경찰,남동구,A씨,정보통신,방송",인천 여고생을 집단폭행하고 성매매를 강요했던 가해자 4명이 경찰에 잡혔다. \n\n...,NaN,20180109,한국경제,NaN,NaN
201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"‘인천 여고생’ 집단 폭행, 성매매 강요 가해자들.. 휴게소서 체포","오산,인천,남동구,남동,부산","경찰서,한경닷컴,간석동,경찰,고속도로순찰대",인천에서 여고생을 감금 및 집단 폭행하고 성매매까지 강요한 가해자들이 붙잡\n혔다....,NaN,20180108,한국경제,NaN,NaN
202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"인천 여고생 집단폭행, 성매매도 강요 가해자 카톡 보니 '섬뜩'","인천,당하","안면,한경닷컴,경찰",인천에서 10대 여고생이 집단 폭행을 당하고 성매매를 강요당해 경찰이 수사에\n 나...,NaN,20180108,한국경제,NaN,NaN


#### features having URL

In [25]:
df_filtered = df[df[df_columns[11]].isna() == False]

In [26]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209 entries, 0 to 271
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   범죄 유형   0 non-null      float64
 1   지역      0 non-null      float64
 2   위도      0 non-null      float64
 3   경도      0 non-null      float64
 4   중복 여부   0 non-null      float64
 5   퀄리티     0 non-null      float64
 6   제외 여부   0 non-null      float64
 7   기사제목    209 non-null    object 
 8   사건 장소   178 non-null    object 
 9   수사 기관   209 non-null    object 
 10  본문      209 non-null    object 
 11  URL     209 non-null    object 
 12  일자      209 non-null    int64  
 13  언론사     209 non-null    object 
 14  기고자     204 non-null    object 
 15  비고      0 non-null      float64
dtypes: float64(8), int64(1), object(7)
memory usage: 27.8+ KB


In [28]:
index_list = df_filtered.index

### web scraping

In [43]:
for index in tqdm(index_list[137:]):
    result = get_html_selenium(url=df.loc[index, "URL"], driver=driver)
    df.loc[index, "selenium_html"] = result


  6%|██████▌                                                                                                                | 4/72 [00:00<00:07,  9.36it/s]

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=124.0.6367.208)
Stacktrace:
0   chromedriver                        0x0000000102f26940 chromedriver + 4368704
1   chromedriver                        0x0000000102f1edd4 chromedriver + 4337108
2   chromedriver                        0x0000000102b42c04 chromedriver + 289796
3   chromedriver                        0x0000000102b3baf8 chromedriver + 260856
4   chromedriver                        0x0000000102b2d5e0 chromedriver + 202208
5   chromedriver                        0x0000000102b2e8d4 chromedriver + 207060
6   chromedriver                        0x0000000102b2d8fc chromedriver + 203004
7   chromedriver                        0x0000000102b2d294 chromedriver + 201364
8   chromedriver                        0x0000000102b2cf5c chromedriver + 200540
9   chromedriver                        0x0000000102b2ad20 chromedriver + 191776
10  chromedriver                        0x0000000102b2b6bc chrome


 14%|████████████████▍                                                                                                     | 10/72 [00:00<00:03, 15.97it/s]

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=124.0.6367.208)
Stacktrace:
0   chromedriver                        0x0000000102f26940 chromedriver + 4368704
1   chromedriver                        0x0000000102f1edd4 chromedriver + 4337108
2   chromedriver                        0x0000000102b42c04 chromedriver + 289796
3   chromedriver                        0x0000000102b3baf8 chromedriver + 260856
4   chromedriver                        0x0000000102b2d5e0 chromedriver + 202208
5   chromedriver                        0x0000000102b2e8d4 chromedriver + 207060
6   chromedriver                        0x0000000102b2d8fc chromedriver + 203004
7   chromedriver                        0x0000000102b2d294 chromedriver + 201364
8   chromedriver                        0x0000000102b2cf5c chromedriver + 200540
9   chromedriver                        0x0000000102b2ad20 chromedriver + 191776
10  chromedriver                        0x0000000102b2b6bc chrome


 19%|██████████████████████▉                                                                                               | 14/72 [00:01<00:03, 17.45it/s]

...


 78%|███████████████████████████████████████████████████████████████████████████████████████████▊                          | 56/72 [00:02<00:00, 21.46it/s]

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=124.0.6367.208)
Stacktrace:
0   chromedriver                        0x0000000102f26940 chromedriver + 4368704
1   chromedriver                        0x0000000102f1edd4 chromedriver + 4337108
2   chromedriver                        0x0000000102b42c04 chromedriver + 289796
3   chromedriver                        0x0000000102b3baf8 chromedriver + 260856
4   chromedriver                        0x0000000102b2d5e0 chromedriver + 202208
5   chromedriver                        0x0000000102b2e8d4 chromedriver + 207060
6   chromedriver                        0x0000000102b2d8fc chromedriver + 203004
7   chromedriver                        0x0000000102b2d294 chromedriver + 201364
8   chromedriver                        0x0000000102b2cf5c chromedriver + 200540
9   chromedriver                        0x0000000102b2ad20 chromedriver + 191776
10  chromedriver                        0x0000000102b2b6bc chrome


 82%|████████████████████████████████████████████████████████████████████████████████████████████████▋                     | 59/72 [00:03<00:00, 20.91it/s]

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=124.0.6367.208)
Stacktrace:
0   chromedriver                        0x0000000102f26940 chromedriver + 4368704
1   chromedriver                        0x0000000102f1edd4 chromedriver + 4337108
2   chromedriver                        0x0000000102b42c04 chromedriver + 289796
3   chromedriver                        0x0000000102b3baf8 chromedriver + 260856
4   chromedriver                        0x0000000102b2d5e0 chromedriver + 202208
5   chromedriver                        0x0000000102b2e8d4 chromedriver + 207060
6   chromedriver                        0x0000000102b2d8fc chromedriver + 203004
7   chromedriver                        0x0000000102b2d294 chromedriver + 201364
8   chromedriver                        0x0000000102b2cf5c chromedriver + 200540
9   chromedriver                        0x0000000102b2ad20 chromedriver + 191776
10  chromedriver                        0x0000000102b2b6bc chrome


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:23<00:00,  3.03it/s]


In [42]:
result = get_html_selenium(url=df.loc[137, "URL"], driver=driver)

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=124.0.6367.208)
Stacktrace:
0   chromedriver                        0x0000000102f26940 chromedriver + 4368704
1   chromedriver                        0x0000000102f1edd4 chromedriver + 4337108
2   chromedriver                        0x0000000102b42c04 chromedriver + 289796
3   chromedriver                        0x0000000102b3baf8 chromedriver + 260856
4   chromedriver                        0x0000000102b2d5e0 chromedriver + 202208
5   chromedriver                        0x0000000102b2e8d4 chromedriver + 207060
6   chromedriver                        0x0000000102b2d8fc chromedriver + 203004
7   chromedriver                        0x0000000102b2d294 chromedriver + 201364
8   chromedriver                        0x0000000102b2cf5c chromedriver + 200540
9   chromedriver                        0x0000000102b2ad20 chromedriver + 191776
10  chromedriver                        0x0000000102b2b6bc chrome

In [61]:
url_length = len(df["URL"])
url_length

377

In [62]:
html_length = len(df["selenium_html"])
html_length

377

In [47]:
df.head(1)

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,selenium_html
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""같이 살자"" 2억 뜯어낸 채팅녀 알고 보니 20대 남자였다",2억여원,"이대,검찰,한경닷컴 객원,서울서부지법,재판부,법원,형사항",채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성...,https://www.hankyung.com/society/article/20220...,20220207,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."


In [50]:
driver.close()
driver.quit()

### html parsing

In [51]:
html = df.loc[0, 'selenium_html']

In [53]:
def save_to_markdown(content, file_path):
    """
    Save content to a Markdown file.

    Parameters:
    - content (str): The content to be saved.
    - file_path (str): The path to the Markdown file.
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

In [54]:
save_to_markdown(html, './test_html.md')

In [49]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [103]:
article_body = soup.find_all("div", class_="article-body")

In [104]:
type(article_body)

bs4.element.ResultSet

In [106]:
article_body[0].get_text()

'\n\n\n\n\nA씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개한 뒤 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어냈다. 사진은 기사와 무관함. /사진=게티이미지뱅크 \n채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성이 항소심에서도 실형을 선고받았다. 7일 법조계에 따르면 서울서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절도·여신전문금융업법 위반·컴퓨터등사용사기 등 혐의로 재판에 넘겨진 A씨(24)에게 1심과 같은 징역 5년을 선고했다.        A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개했다. 그는 남성들에게 교제를 하자거나 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어낸 혐의로 재판에 넘겨졌다. A씨는 피해자 B씨에게 "같이 살 집을 구하자. 보증금, 살림살이를 구입하는데 필요한 돈은 내가 관리하겠다"고 속여 2주 만에 3260만원을 편취했다. 또 다른 피해자 C씨에게는 "사귀자"고 접근해 돈을 빌리고, 계좌번호와 비밀번호 등 개인정보를 건네 받아 은행에서 대신 대출받는 방식으로 총 1730만원을 빼앗았다.        피해자 D씨에게는 음란행위 영상을 받은 뒤 "네가 일하는 곳에 영상을 뿌리겠다"고 협박해 410만원을 갈취하고, 수백만원을 대출받아 빼돌리기도 했다. 1심 재판부는 "범행 수법과 방법 등이 상당히 불량하고, 사회적 비난의 여지가 큰 점 등을 불리한 정상으로 참작한다"면서 징역 5년을 선고했다. 이후 A씨는 형이 너무 무겁다며, 검창른 너무 가볍다며 각각 항소했지만 항소심 재판부의 판단도 다르지 않았다.        항소심 재판부는 "피해자가 수십 명이고, 피해액 합계가 2억4000만원에 이르는데도 피해 회복을 위해 별다른 노력을 하지 않았다"고 지적했다. 이어 "절도 등으로 여러 차례 소년보호처분을 받은 전력이 있고, 재판 중에도 다른 미결수용자를 폭행하는 등 규율위반 행위로 금치 30일 

In [84]:
len(article_body)

0

#### def: get_article_text()

In [107]:
from bs4 import BeautifulSoup

def get_article_text(html:str) -> str:
    soup = BeautifulSoup(html, 'html.parser')
    article_body = soup.find_all("div", class_="article-body")
    if len(article_body) != 0:
        return article_body[0].get_text()
    else:
        return ""

In [70]:
df[df['selenium_html'].notna()]

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,selenium_html
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""같이 살자"" 2억 뜯어낸 채팅녀 알고 보니 20대 남자였다",2억여원,"이대,검찰,한경닷컴 객원,서울서부지법,재판부,법원,형사항",채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성...,https://www.hankyung.com/society/article/20220...,20220207,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'군인비하' 여고생 위문편지도 봉사활동 인정될까?,"서울,서울강서양천교육지원청,여고","국민신문고,서울시교육청,사나,국군,특,서울강서양천교육지원청,별시,교육청",지난달 초 서울 모 여고에서 군인에게 위문 편지를 보내는 과정에서 군인 비하 문구를...,https://www.hankyung.com/society/article/20220...,20220204,한국경제,이미나,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,여고생 등에 '몰래 소변' 30대男 무죄→벌금형 뒤집혔다,충남,"한경닷컴 객원,형사항소,당하,재판부,대법원,대전지법",10대 여학생 뒤에서 몰래 소변을 본 30대 남성이 1 2심에서 추행 혐의 무죄 판...,https://www.hankyung.com/society/article/20220...,20220126,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""'승무원 룩북' 유튜버, 입던 속옷 돈 받고 팔려고 "" 폭로","적나라,아프리카,룩북","미국,유튜브,청구,정보통신,유튜버,대한항공,VVIP",'승무원 룩북' 영상으로 논란이 불거진 유튜버가 자신이 입던 속옷을 나눠준다는 이벤...,https://www.hankyung.com/life/article/20211222...,20211222,한국경제,김소연,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,가출 여고생 '성매매' 강요 20대男 공범 여친 몰래 '성폭행' 하기도,"아동청소년의성보호에관한법률,경기,인천,북구,포항지원","애플,한경닷컴 객원,대구지방법원,재판부,제1형사부,포항시",가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중...,https://www.hankyung.com/society/article/20211...,20211219,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'부평 묻지마 폭행' 영상 확산 17세 女고생도 가담 '충격',"부평,인천시","한경닷컴,부평구,인천부평경찰서",부평 묻지마 폭행 \n \n부평 묻지마 폭행 사건 피의자 가운데 여고생도 포함된 것...,https://www.hankyung.com/society/article/20150...,20150924,한국경제,한예진,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"'부평 묻지마 폭행' 사건, 男女 커플 갑자기 폭행 당해 당시 동영상 보니 '충격'","인천,부평,인천시","한경닷컴,부평경찰서,부평구",'부평 묻지마 폭행' 사건에 여고생이 포함된 것으로 확인됐다. \n \n'부평 묻지...,https://www.hankyung.com/entertainment/article...,20150924,한국경제,한예진,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."
269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"부평 묻지마 폭행 가해자, 신상 강제 공개 네티즌 분노 유발한 글 보니 '충격'","인천,부평,인천시","부평구,한경닷컴,부평경찰서,경찰","부평 묻지마 폭행 가해자, 신상 강제 공개 네티즌 분노 유발한 글 보니 \n \n ...",https://www.hankyung.com/entertainment/article...,20150924,한국경제,김예랑,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."
270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'부평 묻지마 폭행' 사건에 17살 女고생도 가담 당시 동영상 보니 '충격',"인천,부평,인천시","한경닷컴,부평경찰서,부평구",부평 묻지마 폭행 \n \n'부평 묻지마 폭행' 사건에 여고생이 포함된 것으로 확인...,https://www.hankyung.com/society/article/20150...,20150924,한국경제,한예진,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t..."


In [72]:
# df_html = df[(
#     df[df['selenium_html'].notna()]
# ) & (
#     df[df['selenium_html'] != ""]
# )]

df_html = df[
    df['selenium_html'].notna() & 
    (df['selenium_html'] != "")
]


In [73]:
df_html.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148 entries, 0 to 271
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   범죄 유형          0 non-null      float64
 1   지역             0 non-null      float64
 2   위도             0 non-null      float64
 3   경도             0 non-null      float64
 4   중복 여부          0 non-null      float64
 5   퀄리티            0 non-null      float64
 6   제외 여부          0 non-null      float64
 7   기사제목           148 non-null    object 
 8   사건 장소          125 non-null    object 
 9   수사 기관          148 non-null    object 
 10  본문             148 non-null    object 
 11  URL            148 non-null    object 
 12  일자             148 non-null    int64  
 13  언론사            148 non-null    object 
 14  기고자            144 non-null    object 
 15  비고             0 non-null      float64
 16  selenium_html  148 non-null    object 
dtypes: float64(8), int64(1), object(8)
memory usage: 20.8+ KB


In [87]:
index_list = df_html.index
index_list

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       262, 263, 264, 265, 266, 267, 268, 269, 270, 271],
      dtype='int64', length=148)

In [110]:
for index in tqdm(index_list):
    result = get_article_text(html=df.loc[index, "selenium_html"])
    df.loc[index, "content"] = result

  0%|          | 0/148 [00:00<?, ?it/s]

In [91]:
print(index_list[0])
print(type(index_list[0]))

0
<class 'numpy.int64'>


In [94]:
df.loc[int(index_list[0]), "content"] = ""

In [93]:
df.loc[0, "content"] = ""

In [96]:
result = get_article_text(html=df.loc[0, "selenium_html"])

In [98]:
result.get_text()

'\n\n\n\n\nA씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개한 뒤 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어냈다. 사진은 기사와 무관함. /사진=게티이미지뱅크 \n채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성이 항소심에서도 실형을 선고받았다. 7일 법조계에 따르면 서울서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절도·여신전문금융업법 위반·컴퓨터등사용사기 등 혐의로 재판에 넘겨진 A씨(24)에게 1심과 같은 징역 5년을 선고했다.        A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개했다. 그는 남성들에게 교제를 하자거나 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어낸 혐의로 재판에 넘겨졌다. A씨는 피해자 B씨에게 "같이 살 집을 구하자. 보증금, 살림살이를 구입하는데 필요한 돈은 내가 관리하겠다"고 속여 2주 만에 3260만원을 편취했다. 또 다른 피해자 C씨에게는 "사귀자"고 접근해 돈을 빌리고, 계좌번호와 비밀번호 등 개인정보를 건네 받아 은행에서 대신 대출받는 방식으로 총 1730만원을 빼앗았다.        피해자 D씨에게는 음란행위 영상을 받은 뒤 "네가 일하는 곳에 영상을 뿌리겠다"고 협박해 410만원을 갈취하고, 수백만원을 대출받아 빼돌리기도 했다. 1심 재판부는 "범행 수법과 방법 등이 상당히 불량하고, 사회적 비난의 여지가 큰 점 등을 불리한 정상으로 참작한다"면서 징역 5년을 선고했다. 이후 A씨는 형이 너무 무겁다며, 검창른 너무 가볍다며 각각 항소했지만 항소심 재판부의 판단도 다르지 않았다.        항소심 재판부는 "피해자가 수십 명이고, 피해액 합계가 2억4000만원에 이르는데도 피해 회복을 위해 별다른 노력을 하지 않았다"고 지적했다. 이어 "절도 등으로 여러 차례 소년보호처분을 받은 전력이 있고, 재판 중에도 다른 미결수용자를 폭행하는 등 규율위반 행위로 금치 30일 

#### inspection

In [112]:
df.loc[0, 'content']

'\n\n\n\n\nA씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개한 뒤 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어냈다. 사진은 기사와 무관함. /사진=게티이미지뱅크 \n채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성이 항소심에서도 실형을 선고받았다. 7일 법조계에 따르면 서울서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절도·여신전문금융업법 위반·컴퓨터등사용사기 등 혐의로 재판에 넘겨진 A씨(24)에게 1심과 같은 징역 5년을 선고했다.        A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개했다. 그는 남성들에게 교제를 하자거나 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어낸 혐의로 재판에 넘겨졌다. A씨는 피해자 B씨에게 "같이 살 집을 구하자. 보증금, 살림살이를 구입하는데 필요한 돈은 내가 관리하겠다"고 속여 2주 만에 3260만원을 편취했다. 또 다른 피해자 C씨에게는 "사귀자"고 접근해 돈을 빌리고, 계좌번호와 비밀번호 등 개인정보를 건네 받아 은행에서 대신 대출받는 방식으로 총 1730만원을 빼앗았다.        피해자 D씨에게는 음란행위 영상을 받은 뒤 "네가 일하는 곳에 영상을 뿌리겠다"고 협박해 410만원을 갈취하고, 수백만원을 대출받아 빼돌리기도 했다. 1심 재판부는 "범행 수법과 방법 등이 상당히 불량하고, 사회적 비난의 여지가 큰 점 등을 불리한 정상으로 참작한다"면서 징역 5년을 선고했다. 이후 A씨는 형이 너무 무겁다며, 검창른 너무 가볍다며 각각 항소했지만 항소심 재판부의 판단도 다르지 않았다.        항소심 재판부는 "피해자가 수십 명이고, 피해액 합계가 2억4000만원에 이르는데도 피해 회복을 위해 별다른 노력을 하지 않았다"고 지적했다. 이어 "절도 등으로 여러 차례 소년보호처분을 받은 전력이 있고, 재판 중에도 다른 미결수용자를 폭행하는 등 규율위반 행위로 금치 30일 

### save

In [113]:
df.to_parquet(
    './dataframe/flasher_hk_20130101_20220307.gzip',
    compression='gzip')

### class

In [1]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var, get_current_time
from utils.webscraping import ArticleScraper

In [2]:
df = pd.read_excel("./dataframe/EDITED_1_바바리맨_자연어_한국경제_2013.1.1~2022.3.7 (1).xlsx")
df_test = df.iloc[:2]

In [5]:
scr = ArticleScraper(df)

In [6]:
scr.process()

  0%|          | 0/209 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### SettingWithCopyWarning

In [5]:
import pandas as pd

# Sample DataFrame
data = {'some_column': [1, 2, 3]}
df = pd.DataFrame(data)

# Initialize the new column with an empty string
new_column_name = 'content'
df.loc[:, new_column_name] = ''

# Example indexes
indexes = [0, 1, 2]

# Example function to get content
def get_content():
    return "sample content"

# Update the DataFrame with new content
for index in indexes:
    result = get_content()
    df.loc[int(index), new_column_name] = str(result)

print(df)

   some_column         content
0            1  sample content
1            2  sample content
2            3  sample content


# YTN

In [1]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var, get_current_time
from utils.webscraping import ArticleScraper

In [2]:
df = pd.read_excel("./dataframe/EDITED_2_헌팅+미행_자연어_한국경제_2013.1.1~2022.3.7.xlsx")

In [3]:
df_test = df.iloc[:2]

In [4]:
scr = ArticleScraper(df)

In [5]:
scr.process()

  0%|          | 0/101 [00:00<?, ?it/s]

KeyboardInterrupt: 

86/101

#### timeout

In [18]:
import concurrent.futures
import time

def get_count():
    # Simulate a function that may take a long time to process
    sleep(3)  # Sleep for 65 seconds to simulate a long-running process
    return "Count done"

# counts = [1, 2, 3, 4, 5]
counts = [1, 2]

# Define a function to run with a timeout
def run_with_timeout(fn, timeout):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(fn)
        try:
            result = future.result(timeout=timeout)
            return result
        except concurrent.futures.TimeoutError:
            return None

In [19]:
from time import time, sleep

start = time()

for count in counts:
    print(f"Processing count {count}")
    result = run_with_timeout(get_count, 2)
    if result is None:
        print(f"Skipped count {count} due to timeout")
    else:
        print(result)

end = time()

print(end - start)

Processing count 1
Skipped count 1 due to timeout
Processing count 2
Skipped count 2 due to timeout
6.011352062225342


In [21]:
import time
import random
import datetime

class TimeoutException(Exception):
    def __init__(self, *args, **kwargs):
        Exception.__init__(self, *args, **kwargs)

def busy_work():

    # Pretend to do something useful
    time.sleep(random.uniform(0.3, 0.6))

def train_loadbatch_from_lists(batch_size, timeout_sec):

    time_start = datetime.datetime.now()
    batch_xs = []
    batch_ys = []

    for i in range(0, batch_size+1):
        busy_work()
        batch_xs.append(i)
        batch_ys.append(i)

        time_elapsed = datetime.datetime.now() - time_start
        print ('Elapsed:', time_elapsed)
        if time_elapsed > timeout_sec:
            raise TimeoutException()

    return batch_xs, batch_ys

In [24]:
timeout_sec = datetime.timedelta(seconds=5)
batch_size = 10
try:
    print ('Processing batch')
    batch_xs, batch_ys = train_loadbatch_from_lists(batch_size, timeout_sec)
    print ('Completed successfully')
    print (batch_xs, batch_ys)
except TimeoutException as e:
    print ('Timeout after processing N records')

Processing batch
Elapsed: 0:00:00.498515
Elapsed: 0:00:00.836707
Elapsed: 0:00:01.315385
Elapsed: 0:00:01.811561
Elapsed: 0:00:02.329363
Elapsed: 0:00:02.769212
Elapsed: 0:00:03.332679
Elapsed: 0:00:03.716981
Elapsed: 0:00:04.180623
Elapsed: 0:00:04.745530
Elapsed: 0:00:05.202885
Timeout after processing N records


In [31]:
import signal
import time

def timeout_handler(num, stack):
    print("Received SIGALRM")
    raise Exception("FUBAR")

def long_function():
    print("HEAVYYYYYYY WWWWWWWWORKING!!!")
    time.sleep(15)

def set_alarm(duration: int):
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(duration)

In [32]:
try:
    set_alarm(3)
    print("Before: %s" % time.strftime("%M:%S"))
    long_function()
except Exception as ex:
    print(ex)
finally:
    signal.alarm(0)
    print("After: %s" % time.strftime("%M:%S"))

Before: 45:30
HEAVYYYYYYY WWWWWWWWORKING!!!
Received SIGALRM
FUBAR
After: 45:33


#### apply

In [33]:
scr.process()

  0%|          | 0/101 [00:00<?, ?it/s]

WebDriverException: Message: unknown error: net::ERR_CONNECTION_RESET
  (Session info: chrome=125.0.6422.77)
Stacktrace:
0   chromedriver                        0x0000000102856510 chromedriver + 4302096
1   chromedriver                        0x000000010284ee58 chromedriver + 4271704
2   chromedriver                        0x000000010248019c chromedriver + 278940
3   chromedriver                        0x0000000102479510 chromedriver + 251152
4   chromedriver                        0x000000010246b120 chromedriver + 192800
5   chromedriver                        0x000000010246c480 chromedriver + 197760
6   chromedriver                        0x000000010246b43c chromedriver + 193596
7   chromedriver                        0x000000010246aad0 chromedriver + 191184
8   chromedriver                        0x000000010246aa7c chromedriver + 191100
9   chromedriver                        0x000000010246885c chromedriver + 182364
10  chromedriver                        0x00000001024691dc chromedr

In [34]:
scr.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   범죄 유형          0 non-null      float64
 1   지역             0 non-null      float64
 2   위도             0 non-null      float64
 3   경도             0 non-null      float64
 4   중복 여부          0 non-null      float64
 5   퀄리티            0 non-null      float64
 6   제외 여부          0 non-null      float64
 7   기사제목           137 non-null    object 
 8   사건 장소          119 non-null    object 
 9   수사 기관          137 non-null    object 
 10  본문             137 non-null    object 
 11  URL            101 non-null    object 
 12  일자             137 non-null    int64  
 13  언론사            137 non-null    object 
 14  기고자            111 non-null    object 
 15  비고             0 non-null      float64
 16  selenium_html  137 non-null    object 
dtypes: float64(8), int64(1), object(8)
memory usage: 18.3+

# end